# Rapport sur la comparaison entre le modèle générique et le modèle spécialisé

Dans ce notebook, nous effectuons une comparaison entre le modèle GPT-2 générique et le modèle GPT-2 spécialisé, en comparant la génération de texte sur des prompts techniques spécifiques, et en effectuant des tests d'évaluation en se basant sur le jeu de données d'articles scientifiques.

L'objectif est de pouvoir d'observer la différence en termes de qualité et de précision des réponse du modèle par rapport à des questions/prompts techniques et scientifiques, entre le modèle générique et spécialisé.

**Groupe ALOQAS**

- Aurélien ZUFIC
- Lucas AGUETAÏ
- Ony ANDRIATSAHAVOJAONA
- Quentin VERMEERSCH
- Alexandre HUYNH
- Samuel DORISMOND


---
## Import de packages

In [1]:
!pip install keras_nlp -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
import tensorflow as tf
from google.colab import drive
import json
import os
import sys
import keras_nlp
import time
from tensorflow import keras
import numpy as np

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/package')
#import datasets_scientific_paper as load_ds

Using TensorFlow backend
Mounted at /content/drive


In [3]:
from datasets import load_dataset

dataset = load_dataset("scientific_papers", "pubmed")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

---
## Initialisation du modèle GPT-2

In [4]:
os.environ["KERAS_BACKEND"] = "tensorflow"

keras.mixed_precision.set_global_policy("mixed_float16")

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 299kB/s]
100%|██████████| 446k/446k [00:00<00:00, 2.94MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 4.84MB/s]
100%|██████████| 484/484 [00:00<00:00, 488kB/s]
100%|██████████| 475M/475M [00:05<00:00, 86.5MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


---
## Génération de texte avec le modèle GPT-2 générique (sans fine-tuning)

In [5]:
# Fonction génération de texte
def generate_text(prompt):
    start = time.time()
    output = gpt2_lm.generate(prompt, max_length=200)
    end = time.time()
    print('________________________________________________')
    print(f"\nPrompt: {prompt}")
    print("GPT-2 output:")
    print(output)
    print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")
    #return output

In [6]:
# Prompts génération de texte
prompts = [
    "Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that", # extrait article repository arxiv
    "How does the structure of a multiplex affects the walk applications to real-world airline transportation networks ?", # extrait article repository arxiv
    "Could you explain to me what is the Inherited defect of coagulation factors and PE Inherited defects of fibrinolysis ?", # extrait article repository pubmed
    "What are the latest advancements in cancer research ?", # exemple suggestion personnelle
    "What is the impact of diet on heart disease according to recent studies ?", # exemple suggestion personnelle
    "What are the usual causes of lung pain ?", # exemple suggestion personnelle
    "", # génération vide
    "" # génération vide
]

In [ ]:
# Génération de texte avec le modèle générique

print("Output GPT-2 générique")
i=0
while i < len(prompts):
  generate_text(prompts[i])
  i+=1

Output GPT-2 générique
________________________________________________

Prompt: Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that
GPT-2 output:
Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that the first step in the process of hypertrophy is the removal of the sympathetic nervous system from the heart, and this process involves the removal of the blood-brain barrier (BBB), the main source of the blood supply to the central nervous system. The BBB is a central component of the heart and, therefore, is the primary organ of circulation for the heart.

The heart is a large body of blood, with many different types, and the BBB, in particular the blood supply to the heart, is a major organ of the central nervous system (CNS), which is responsible for the flow of blood to t

'\nprint(output_generique0 + "\n")\nprint(output_generique1 + "\n")\nprint(output_generique2 + "\n")\n'

In [7]:
print("Output GPT-2 générique - Prompt string vide")
generate_text("")
generate_text("")

Output GPT-2 générique - Prompt string vide
________________________________________________

Prompt: 
GPT-2 output:
"I don't think you should be able to go on the Internet, and that's why you're doing it. It's not a good idea," he said.

The former Republican senator from Texas was among a handful of lawmakers who have taken to Twitter to express their support for Mr. Trump's travel ban.

"The people of the U.S. have a right to know who the president is and what his intentions are," said Sen. John McCain, R-Ariz., who was among those who took to the platform to call for "an independent and impartial investigation" of the ban.

The president's executive order temporarily blocked immigration from seven Muslim-majority countries, including Iran, Iraq, Libya, Somalia, Sudan, Syria and Yemen.

The ban is a response to a series of high-profile incidents in recent years that have raised questions about whether President Trump's executive actions will have a negative effect on U.S. security. 

---
## Test d'évaluation du modèle GPT-2 générique (sans fine-tuning)

In [ ]:
val_features = dataset["validation"][:]["article"]

# Évaluer le modèle sur les données de validation
with tf.device('/device:GPU:0'):
  val_loss, val_accuracy = gpt2_lm.evaluate(
      x=val_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 générique - Validation Loss: {val_loss}")
print(f"GPT-2 générique - Validation Accuracy: {val_accuracy}")

208/208 [==============================] - 3346s 16s/step - loss: 3.9689 - sparse_categorical_accuracy: 0.3391
Validation Loss: 3.9688880443573
Validation Accuracy: 0.33905521035194397


In [ ]:
test_features = dataset["test"][:]["article"]

# Évaluer le modèle sur les données de test
with tf.device('/device:GPU:0'):
  test_loss, test_accuracy = gpt2_lm.evaluate(
      x=np.array(test_features),
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 générique - Test Loss: {test_loss}")
print(f"GPT-2 générique - Test Accuracy: {test_accuracy}")

209/209 [==============================] - 3310s 16s/step - loss: 3.9504 - sparse_categorical_accuracy: 0.3406
GPT-2 spécialisé - Test Loss: 3.9503707885742188
GPT-2 spécialisé - Test Accuracy: 0.34061935544013977


---
## Génération de texte avec le modèle GPT-2 spécialisé (avec fine-tuning)

Finetuné sur le jeu de données d'articles scientifiques.

In [12]:
# Chargement des paramètres GPT-2 finetuné sur les articles scientifiques
#checkpoint_path = "/content/drive/MyDrive/training_data_all_remake/cp.ckpt"
checkpoint_path = "/content/drive/MyDrive/training_data_all_remake_base_epochs/cp.ckpt"

gpt2_lm.load_weights(checkpoint_path)

### Modèle GPT-2 finetuné - 1ère version

GPT-2 Base, entrainé sur dataset téléchargé et divisé manuellement

In [ ]:
# Génération de texte avec le modèle spécialisé

print("Output GPT-2 spécialisé")
i = 0
while i < len(prompts):
  generate_text(prompts[i])
  i+=1

Output GPT-2 spécialisé
________________________________________________

Prompt: Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that
GPT-2 output:
Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that there is a growing body of evidence indicating the association between cardiometabolic changes and cardiovascular disease . the relationship between cardiometabolic and cardiovascular risk is well established.2 the role of the heart is to support the development of a cardiometabolic state , and the role of the heart is to support the development of a cardiometabolic state . the heart is a complex organ and its function requires the ability to maintain and regenerate its integrity , and the heart is the
TOTAL TIME ELAPSED: 32.71s
_______________________________________________

'\nprint(output_specialise0 + "\n")\nprint(output_specialise1 + "\n")\nprint(output_specialise2 + "\n")\n'

### Modèle GPT-2 finetuné - 2ème version

GPT-2 base, entrainé sur dataset importé de Hugging Face, identique à celle de Tensor Flow

In [9]:
# Génération de texte avec le modèle spécialisé

print("Output GPT-2 spécialisé")
i = 0
while i < len(prompts):
  generate_text(prompts[i])
  i+=1

Output GPT-2 spécialisé
________________________________________________

Prompt: Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that
GPT-2 output:
Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that a number of studies suggest that hypertrophy of adipose tissue ( at ) is the cause of at , and that this is associated with increased risk of coronary events .
these findings are supported by the evidence that the metabolic rate ( mr ) is elevated during the first 2 weeks of atherosclerosis and that the mr is elevated in patients with coronary artery disease ( cad ) and that the mr is elevated in patients with non - cad [ 2 , 3 ] .
TOTAL TIME ELAPSED: 1.73s
________________________________________________

Prompt: How does the structure of a multiplex affects the walk applica

**Observations**

Nous observons certains défauts de génération, tels que lors de la génération avec le prompt "What are the latest advancements in cancer research ?" ayant une phrase non complétée.

Le prompt "What are the usual causes of lung pain ?" semble bien répondre à la question en citant une des causes de douleurs pulmonaires, même si des répétitions peuvent être soulignés.

Un constat peut être fait concernant les prompts vides, où le texte généré est très orienté vers ceux du jeu de données, pouvant s'apparenter quasimment à un extrait d'articles scientifiques.

### Modèle GPT-2 finetuné - 3ème version

GPT-2 base, entrainé en 2 fois (50/50% articles train)sur dataset importé de Hugging Face, avec 3 epochs pour chaque parties entrainés.

In [13]:
# Génération de texte avec le modèle spécialisé

print("Output GPT-2 spécialisé")
i = 0
while i < len(prompts):
  generate_text(prompts[i])
  i+=1

Output GPT-2 spécialisé
________________________________________________

Prompt: Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that
GPT-2 output:
Cardiac hypertrophy is initiated as an adaptive response to sustained overload but progresses pathologically as heart failure ensues. On this topic, I know that the most common cause of heart failure is myocardial infarction ( mi) .
it occurs when a large number of myocardial cells die in the event of a cardiac event , and the loss of cells is accompanied by an increase in intracellular calcium concentration .
this is a major complication of mi , with the incidence of myocardial infarction ( mi ) increasing with advancing age . in older age , the incidence of mi
is about 10 times
TOTAL TIME ELAPSED: 0.58s
________________________________________________

Prompt: How does the structure of a multiplex affects the walk applications to r

**Observations**

Nous observons ici que les résultats sont nettement meilleurs par rapport aux autres modèles.

Avec le prompt "What is the impact of diet on heart disease according to recent studies ?", le modèle évoque bien certaines causes précises avec des termes techniques tels que "saturated fat and low - saturated fatty acids ( lfas )"

Nous avons même un cas de génération ressemblant beaucoup à une vraie interaction question-réponse avec "What are the usual causes of lung pain ?", donnant comme réponse "
the most common cause of lung pain is bronchial asthma , with a lifetime prevalence of 2%" (Quelles sont les causes des douleurs pulmonaires ? -> La cause la plus commune de douleur pulmonaire est...).

---
## Test d'évaluation du modèle GPT-2 spécialisé (avec fine-tuning)

Finetuné sur le jeu de données d'articles scientifiques.

### Modèle GPT-2 finetuné - 2ème version

GPT-2 base, entrainé sur dataset importé de Hugging Face, identique à celle de Tensor Flow

In [10]:
val_features = dataset["validation"][:]["article"]

# Évaluer le modèle sur les données de validation
with tf.device('/device:GPU:0'):
  val_loss, val_accuracy = gpt2_lm.evaluate(
      x=val_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 spécialisé - Validation Loss: {val_loss}")
print(f"GPT-2 spécialisé - Validation Accuracy: {val_accuracy}")

208/208 [==============================] - 209s 939ms/step - loss: 2.8854 - sparse_categorical_accuracy: 0.4459
GPT-2 spécialisé - Validation Loss: 2.885439872741699
GPT-2 spécialisé - Validation Accuracy: 0.44592615962028503


In [11]:
test_features = dataset["test"][:]["article"]

# Évaluer le modèle sur les données de test
with tf.device('/device:GPU:0'):
  test_loss, test_accuracy = gpt2_lm.evaluate(
      x=test_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 spécialisé - Test Loss: {test_loss}")
print(f"GPT-2 spécialisé - Test Accuracy: {test_accuracy}")

209/209 [==============================] - 207s 977ms/step - loss: 2.8649 - sparse_categorical_accuracy: 0.4483
GPT-2 spécialisé - Test Loss: 2.86492657661438
GPT-2 spécialisé - Test Accuracy: 0.44825422763824463


### Modèle GPT-2 finetuné - 3ème version

GPT-2 base, entrainé en 2 fois (50/50% articles train)sur dataset importé de Hugging Face, avec 3 epochs pour chaque parties entrainés.

In [14]:
val_features = dataset["validation"][:]["article"]

# Évaluer le modèle sur les données de validation
with tf.device('/device:GPU:0'):
  val_loss, val_accuracy = gpt2_lm.evaluate(
      x=val_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 spécialisé - Validation Loss: {val_loss}")
print(f"GPT-2 spécialisé - Validation Accuracy: {val_accuracy}")

208/208 [==============================] - 201s 949ms/step - loss: 2.7834 - sparse_categorical_accuracy: 0.4607
GPT-2 spécialisé - Validation Loss: 2.783418655395508
GPT-2 spécialisé - Validation Accuracy: 0.46072784066200256


In [15]:
test_features = dataset["test"][:]["article"]

# Évaluer le modèle sur les données de test
with tf.device('/device:GPU:0'):
  test_loss, test_accuracy = gpt2_lm.evaluate(
      x=test_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"GPT-2 spécialisé - Test Loss: {test_loss}")
print(f"GPT-2 spécialisé - Test Accuracy: {test_accuracy}")

209/209 [==============================] - 217s 1s/step - loss: 2.7643 - sparse_categorical_accuracy: 0.4631
GPT-2 spécialisé - Test Loss: 2.764336347579956
GPT-2 spécialisé - Test Accuracy: 0.4630672335624695


---
## Observations et remarques concernant la génération de texte

On peut remarquer que les réponses du modèle GPT-2 Générique sont vagues, répétitives, et manquent de clarté. Elle ne fournit pas d'informations utiles ou précises sur le sujet. Alors que les réponses modèle GPT-2 Spécialisé sont légèrement plus structurées et semblent se concentrer davantage sur le sujet, bien qu'elle reste encore vague et manque de détails spécifiques.

Sur la question des avancées récentes sur le cancer, le générique se concentre sur des statistiques générales plutôt que sur les avancées spécifiques dans la recherche sur le cancer. Elle manque de détails pertinents sur le sujet, alors que le spécialisé mentionne l'évolution des thérapies et des technologies, ce qui est plus pertinent, mais elle ne fournit pas d'exemples concrets ou de détails spécifiques sur les avancées.

Sur la question de l'impact de l'alimentation sur les maladies cardiaques, la réponse du générique est contradictoire et peut induire en erreur, en particulier en ce qui concerne les graisses saturées. Alors que la réponse du spécialiste aborde les compléments alimentaires et leurs effets sur les maladies cardiaques de manière plus nuancée, bien qu'elle ne soit pas entièrement claire ou concluante.

Sur la question des causes habituelles de la douleur pulmonaire, le générique fournit des informations incorrectes et non pertinentes, telles que la mention du syndrome de mort subite. Le spécialisé se concentre sur des causes plus plausibles telles que l'inflammation bronchique, bien qu'il manque de profondeur dans l'explication.

Nous constatons en particulier qu'une génération de texte effectué sur un prompt vide via le modèle spécialisé oriente bien le sujet vers une thématique scientifique / médicale, démontrant que le modèle a bien pris en compte l'entrainement par fine-tuning.

En se basant sur les réponses obtenus pour le modèle spécialisé, nous constatons que les réponses

Certains défauts peuvent être relevés, tels que l'absence de majuscule en début de phrase ou encore la présence d'espaces avant les signes de ponctuation, rendant les réponses moins lisibles.

On peut donc en conclure que le modèle GPT-2 spécialisé semble offrir des réponses légèrement plus structurées et centrées sur les sujet traités dans les articles scientifques que celui générique, avec des termes scientifiques et des explications plus techniques.<br /> Cependant, même dans le cas du modèle spécialisé, les réponses manquent souvent de détails spécifiques et cohérents, de précision et de clarté.<br />
Nous pouvons tout de même souligner le fait que les réponses deviennent de plus en plus cohérentes et précises au fur et à mesure que nous optimisons le modèle, et paraissent réellement comme des vrais réponses aux questions. Cela est surtout observable avec le modèle fine-tuné avec GPT-2 base avec 3 epochs, où les réponses sont plus précises, cohérentes et bien construites.

---
## Observations et remarques concernant les tests d'évaluation des modèles

En effectuant des tests d'évaluation avec le jeu de données d'articles scientifiques, spécifiquement les parties d'évaluation et de test, nous avons pu constaté une augmentation de précision (accuracy) et une diminution en erreurs (loss) entre le modèle GPT-2 par défaut et celui fine-tuné. Cela signifie que le fine-tuning réalisé a bien eu un impact sur les poids du modèle, et les a plus orientés vers les thématiques, le vocabulaire et le style d'écriture des articles scientifiques.

- **GPT-2 générique**
  - Loss validation : ~3.97
  - Loss test : ~3.95
  - Accuracy validation : ~0.34 (34%)
  - Accuracy test : ~0.34 (34%)
- **GPT-2 spécialisé - modèle base avec 1 epochs par entrainement**
  - Loss validation : ~2.88
  - Loss test : ~2.86
  - Accuracy validation : ~0.44 (44%)
  - Accuracy test : ~0.45 (45%)
- **GPT-2 spécialisé - modèle base avec 3 epochs par entrainement**
  - Loss validation : ~2.78
  - Loss test : ~2.76
  - Accuracy validation : ~0.46 (46%)
  - Accuracy test : ~0.46 (46%)

On constate un grand écart de précision (~10 % de différence) et de perte/échec (env. 1,10 de différence) entre le modèle générique et les modèles spécialisés. Entre les modèles spécialisés, la différence est moins importante mais reste progressive en qualité.

Elle souligne donc l'impact positive du fine-tuning sur le modèle GPT-2, orientant les poids du modèle vers des réponses similaires aux divers articles scientifiques du jeu de données.

---
## Objectifs pour optimiser le modèle

Par la suite, nous envisageons d'optimiser le modèle en essayant d'utiliser des modèles GPT-2 plus performants, avec plus de couches en réseaux de neurones, ainsi que de paramètres. Comme il a été employé dans ce rapport, nous expérimentons et menons plusieurs entrainement différents sur différents niveaux de modèles (gpt2 base, medium, large), et des paramètres différents (différence epochs 1 et 3).

A l'avenir, un des objectifs serait de mener des recherches pour améliorer la génération de texte, comme par exemple en évitant les cas de répétition, ou encore la rapidité d'exécution du modèle.

Nous étudions certaines pistes, notamment avec le "Parameter-efficient fine-tuning" du modèle GPT-2 avec LoRA (Low-Rank Adaptation), qui pourraient réduire le temps d'entrainement et la consommation GPU en conservant la qualité des réponses, ce qui pourrait nous permettre d'envisager des modèles plus importants comme GPT-2 large.